# Парсинг musik-produktiv.com

Данные для проекта будут собраны с сайта musik-produktiv.com - немецкого интернет-магазина музыкальных инструментов. Для парсинга использована библиотека bs4 и ее модуль BeautifulSoup

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import copy

## Парсинг

Посмотрим на код первой страницы сайта.

In [2]:
response = requests.get('https://www.musik-produktiv.com/ru/bas-gitara/')
soup = BeautifulSoup(response.text)
#Выод на экран ограничен в этой и последующих ячейках ограничен по длине для читаемости ноутбука 
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="ru-RU">
 <head>
  <title>
   Бас-гитара | Musik Produktiv
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta charset="utf-8"/>
  <link as="style" href="https://sc1.musik-produktiv.com/css/mp-desktop-20220524-220658.min.css" rel="preload"/>
  <link href="https://sc1.musik-produktiv.com/css/mp-desktop-20220524-220658.min.css" rel="stylesheet"/>
  <link href="https://fonts.googleapis.com/css?family=Source+Sans+Pro:400,600&amp;subset=cyrillic&amp;display=block" rel="stylesheet"/>
  <style>
   .mp-sprite { background-image: url('https://sc1.musik-produktiv.com/img/css-sprites/online-shop-icons-20220226.png');}
  </style>
  <link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <script defer="" src="https://sc1.musik-produktiv.com/js/mp-desktop-20220524-220658.min.js">
  </script>
  <style>
   .mp-sprite-sp { background-image: url('https://sc1.musik-produktiv.com/img/css-sprites/online-shop-sp-2018-12-19-ru.pn

Для того чтобы получить данные об инструментах не только с первой страницы раздела "бас-гитары", но и с остальных, необходимо получить список ссылок на каждую страницу сайта. Контент раздела занимает 10 страниц, но на первой странице отображены страницы 1,2,3,4,5...10. 

Найдем в коде страницы ссылки на эти страницы сайта и дополним их недостающими страницами 6,7,8,9.

In [3]:
response_pages = ['https://www.musik-produktiv.com/ru/bas-gitara/']
for i in range(len(soup.select('div.list_page a'))):
    response_pages.append(soup.select('div.list_page a')[i]['href'])
response_pages

['https://www.musik-produktiv.com/ru/bas-gitara/',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=2',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=3',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=4',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=5',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=11']

In [4]:
response_pages = response_pages[:5]
links = ['https://www.musik-produktiv.com/ru/bas-gitara/?p=6',
        'https://www.musik-produktiv.com/ru/bas-gitara/?p=7',
        'https://www.musik-produktiv.com/ru/bas-gitara/?p=8',
        'https://www.musik-produktiv.com/ru/bas-gitara/?p=9',
        'https://www.musik-produktiv.com/ru/bas-gitara/?p=10']
for link in links:
    response_pages.append(link)
response_pages

['https://www.musik-produktiv.com/ru/bas-gitara/',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=2',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=3',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=4',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=5',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=6',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=7',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=8',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=9',
 'https://www.musik-produktiv.com/ru/bas-gitara/?p=10']

Список страниц готов. Пройдемся по каждой странице и вытащим из них названия бас-гитар. Названия находятся в классe "main-right-4col" внутри ограничителей 'li', 'b'

In [5]:
bass_name_list = []
for link in response_pages:
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    for bass in range(len(soup.select('div.main-right-4col li'))):
        bass_name_list.append(soup.select('div.main-right-4col li b')[bass].text)
bass_name_list[:5]

['Höfner Ignition Club Bass VSB',
 'Höfner Violin Ignition Bass SB',
 'Ibanez miKro GSRM20 WNF',
 'ESP LTD B-205 SM STBLKS',
 'Ibanez Soundgear SR300E-SVM']

Аналогично получим цены и ссылки на страницы продуктов.

In [6]:
price_list = []
for link in response_pages:
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    for price in range(len(soup.select('div.main-right-4col li i'))):
        price_list.append(soup.select('div.main-right-4col li i')[price].text)
price_list[:5]

['₽ 25 388', '₽ 24 940', '₽ 14 711', '₽ 50 432', '₽ 25 316']

Из цен уберем пробелы и знаки "₽".

In [7]:
price_list_stripped = []
for price in price_list:
    price = price.replace('₽ ', '')
    price = price.replace(' ', '')
    price_list_stripped.append(price)
price_list = price_list_stripped
price_list[:5]

['25388', '24940', '14711', '50432', '25316']

In [8]:
bass_links_list = []
for link in response_pages:
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    for bass_link in range(len(soup.select('div.main-right-4col li'))):
        bass_links_list.append(soup.select('div.main-right-4col li a')[bass_link]['href'])
bass_links_list[:5]

['https://www.musik-produktiv.com/ru/hoefner-ignition-club-bass-vsb.html',
 'https://www.musik-produktiv.com/ru/hoefner-violin-ignition-bass-sb.html',
 'https://www.musik-produktiv.com/ru/ibanez-mikro-gsrm20-wnf.html',
 'https://www.musik-produktiv.com/ru/esp-ltd-b-205-sm-stblks.html',
 'https://www.musik-produktiv.com/ru/ibanez-soundgear-sr300e-svm.html']

Теперь имея список ссылок на страницы продуктов зайдем на каждую из них и вытащим оттуда описания товаров. Посмотрим на случайную страницу товара и найдем в ней описание:

In [9]:
response = requests.get('https://www.musik-produktiv.com/ru/sandberg-california-tt4-mblk-rmn.html')
soup = BeautifulSoup(response.text)
str(soup)[18000:20000]

'itemscope="" itemtype="http://schema.org/ListItem"><a href="https://www.musik-produktiv.com/ru/bas-gitara/sandberg/" itemprop="item" title="Sandberg"><span itemprop="name">Sandberg</span></a><meta content="3" itemprop="position"/></span><span itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem"><span itemprop="name">California TT4 MBLK RMN</span><meta content="https://www.musik-produktiv.com/ru/sandberg-california-tt4-mblk-rmn.html" itemprop="item"/><meta content="4" itemprop="position"/></span></div>\n<div>\n<div class="main-left-4col">\n<div class="product-image h-500">\n<img alt="Бас-гитара  Sandberg California TT4 MBLK RMN" height="480" src="https://sc1.musik-produktiv.com/img/clips/image-coming-soon-l.png" title="Бас-гитара  Sandberg California TT4 MBLK RMN" width="480"/></div>\n<div id="zoom-big"></div>\n<div class="product-info more-slider">\n<div class="more-slider-content h-500">\n<a class="markenlogo" href="https://www.musik-produktiv.com/ru/sandberg

Описание товара находится в классе "eigenschaften" внутри ограничителей "li". Пройдемся по каждому товару и вытащим описания всех товаров.

In [10]:
description = []
for link in bass_links_list:
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    desc = soup.select('ul.eigenschaften li')
    for i in range(len(desc)):
        desc[i] = desc[i].text
    description.append(desc)
description[:3]

[['Форма корпуса: Битлз бас',
  'Материал корпуса: Клён (мэйпл)',
  'Дека: Ель',
  'Конструкция: Полуакустическая (Семи Холлоу Бади)',
  'Цвет: Винтэйдж Санберст',
  'Конструкция грифа: Закреплён винтами',
  'Мензура: Короткая',
  'Длина мензуры: 30,5" (777 мм)',
  'Гриф: Трёхслойный клён/бук',
  'Накладка грифа: Палисандр (розовое дерево)',
  'Количество ладов: 22',
  'Маркеры (вкладыши) грифа: Классические круглые',
  'Число струн: четырёхструнная',
  'Типы звукоснимателей: Хамбакер',
  'Переключатель звукоснимателей: Переключатель типа слайд',
  'Цвет фурнитуры: Никель',
  'Страна производства: КНР'],
 ['Форма корпуса: Битлз бас',
  'Материал корпуса: Клён (мэйпл)',
  'Дека: Ель',
  'Конструкция: Полуакустическая (Семи Холлоу Бади)',
  'Цвет: Винтэйдж Санберст',
  'Конструкция грифа: Вклеен в корпус',
  'Мензура: Короткая',
  'Длина мензуры: 30,5" (777 мм)',
  'Гриф: Трёхслойный клён/бук',
  'Накладка грифа: Палисандр (розовое дерево)',
  'Количество ладов: 22',
  'Маркеры (вкладыши

Убедимся, что признаки получены для всех товаров.

In [11]:
print(len(bass_name_list))
print(len(price_list))
print(len(bass_links_list))
print(len(description))

600
600
600
600


## Приведение данных к табличному виду

Чтобы получить таблицу, в которой каждый признак будет соответствовать нужному инструменту, приведем полученные данные в датафрейму, используя библиотеку "Pandas".
Так как не каждое описание товара содержит все возможные признаки, необходимо провести подготовительную работу с описаниями товаров. Признаки и значения признака разделены двоеточием - разобьем их в разные колонки и посмотрим на то, насколько данные полные.

In [12]:
features = []
for bass in description:
    for item in bass:
        feature, sep, value = item.partition(':')
        features.append(feature)
features = pd.Series(features)

In [13]:
features.value_counts()

Число струн                                   598
Цвет                                          595
Конструкция                                   588
Конструкция грифа                             588
Мензура                                       587
Длина мензуры                                 586
Гриф                                          586
Количество ладов                              583
Накладка грифа                                581
Материал корпуса                              580
Цвет фурнитуры                                579
Форма корпуса                                 569
Маркеры (вкладыши) грифа                      543
Типы звукоснимателей                          536
Бриджевый звукосниматель                      518
Страна производства                           474
Звукосниматель грифа                          461
Встроенная электроника                        406
Элементы управления                           373
В комплекте поставки                          308


Не все признаки несут ценность в определении стоимости - уберем ненужные. Список всех возможных значений сохраним в "features".

In [14]:
features = list(features.unique())

unnecessary_values = ['Цвет',
             'Цвет фурнитуры',
             'Маркеры (вкладыши) грифа',
             'Элементы управления',
             'В комплекте поставки',
             'Переключатель звукоснимателей',
             'Отличительные особенности',
             'Толщина струн, установленных изготовителем',
             'Вес',
             'Серийный номер униката',
             'Поверхность']

for value in unnecessary_values:
    features.remove(value)

features

['Форма корпуса',
 'Материал корпуса',
 'Дека',
 'Конструкция',
 'Конструкция грифа',
 'Мензура',
 'Длина мензуры',
 'Гриф',
 'Накладка грифа',
 'Количество ладов',
 'Число струн',
 'Типы звукоснимателей',
 'Страна производства',
 'Звукосниматель грифа',
 'Бриджевый звукосниматель',
 'Встроенная электроника',
 'Средний звукосниматель']

In [15]:
for i in range(len(features)):
    for bass in description:
        for item in bass:
            feature, sep, value = item.partition(':')
            if feature not in features:
                bass.remove(item)

В "features_bass" добавим названия признаков каждой бас-гитары, а в "values_bass" добавим соответствующие значения признаков.

In [16]:
features_bass = []
values_bass = []
for bass in range(len(description)):
    features_bass.append([])
    values_bass.append([])
    for i in range(len(description[bass])):
        feature, sep, value = description[bass][i].partition(': ')
        features_bass[bass].append(feature)
        values_bass[bass].append(value)

In [17]:
features_bass[:3]

[['Форма корпуса',
  'Материал корпуса',
  'Дека',
  'Конструкция',
  'Конструкция грифа',
  'Мензура',
  'Длина мензуры',
  'Гриф',
  'Накладка грифа',
  'Количество ладов',
  'Число струн',
  'Типы звукоснимателей',
  'Страна производства'],
 ['Форма корпуса',
  'Материал корпуса',
  'Дека',
  'Конструкция',
  'Конструкция грифа',
  'Мензура',
  'Длина мензуры',
  'Гриф',
  'Накладка грифа',
  'Количество ладов',
  'Число струн',
  'Типы звукоснимателей',
  'Звукосниматель грифа',
  'Бриджевый звукосниматель',
  'Страна производства'],
 ['Форма корпуса',
  'Материал корпуса',
  'Конструкция',
  'Конструкция грифа',
  'Мензура',
  'Длина мензуры',
  'Гриф',
  'Накладка грифа',
  'Количество ладов',
  'Число струн',
  'Типы звукоснимателей',
  'Страна производства']]

In [18]:
values_bass[:3]

[['Битлз бас',
  'Клён (мэйпл)',
  'Ель',
  'Полуакустическая (Семи Холлоу Бади)',
  'Закреплён винтами',
  'Короткая',
  '30,5" (777 мм)',
  'Трёхслойный клён/бук',
  'Палисандр (розовое дерево)',
  '22',
  'четырёхструнная',
  'Хамбакер',
  'КНР'],
 ['Битлз бас',
  'Клён (мэйпл)',
  'Ель',
  'Полуакустическая (Семи Холлоу Бади)',
  'Вклеен в корпус',
  'Короткая',
  '30,5" (777 мм)',
  'Трёхслойный клён/бук',
  'Палисандр (розовое дерево)',
  '22',
  'четырёхструнная',
  'Соупбар',
  'Хофнер Игнишн Стэйпл Никел',
  'Хофнер Игнишн Стэйпл Никел',
  'КНР'],
 ['Саундджиэр',
  'Окоуме',
  'Солид Бади',
  'Закреплён винтами',
  'Короткая',
  '28,63" (727 мм)',
  'Клён (мэйпл)',
  'Древесина новозеландской пинии',
  '22',
  'четырёхструнная',
  '1 x P-Стайл + 1 x J-Стайл',
  'Индонезия']]

Представим список всех возможных признаков "features" в виде датафрейма "features_df"

In [19]:
features_df = pd.DataFrame({'features_bass' : features})
features_df

,features_bass
0,Форма корпуса
1,Материал корпуса
2,Дека
3,Конструкция
4,Конструкция грифа
5,Мензура
6,Длина мензуры
7,Гриф
8,Накладка грифа
9,Количество ладов


Для каждого из 600 инструментов создадим датафрейм, в котором в левом столбце окажутся все возможные признаки (соответствуют списку features_df), а в правом - все значения конкретного инструмента. Там, где признаку не будет соответствовать никакое значение - будет значение "Nan". И заменим значения в values_bass правым столбцом датафрейма.

In [20]:
for i in range(len(values_bass)):
    df = pd.DataFrame({'features_bass' : features_bass[i], 'values_bass' : values_bass[i]})
    df = pd.merge(features_df, df, how = 'outer')
    values_bass[i].clear()
    values_bass[i].append(list(df['values_bass']))
for i in range(len(values_bass)):
    values_bass[i] = values_bass[i][0]

In [21]:
values_bass[:3]

[['Битлз бас',
  'Клён (мэйпл)',
  'Ель',
  'Полуакустическая (Семи Холлоу Бади)',
  'Закреплён винтами',
  'Короткая',
  '30,5" (777 мм)',
  'Трёхслойный клён/бук',
  'Палисандр (розовое дерево)',
  '22',
  'четырёхструнная',
  'Хамбакер',
  'КНР',
  nan,
  nan,
  nan,
  nan],
 ['Битлз бас',
  'Клён (мэйпл)',
  'Ель',
  'Полуакустическая (Семи Холлоу Бади)',
  'Вклеен в корпус',
  'Короткая',
  '30,5" (777 мм)',
  'Трёхслойный клён/бук',
  'Палисандр (розовое дерево)',
  '22',
  'четырёхструнная',
  'Соупбар',
  'КНР',
  'Хофнер Игнишн Стэйпл Никел',
  'Хофнер Игнишн Стэйпл Никел',
  nan,
  nan],
 ['Саундджиэр',
  'Окоуме',
  nan,
  'Солид Бади',
  'Закреплён винтами',
  'Короткая',
  '28,63" (727 мм)',
  'Клён (мэйпл)',
  'Древесина новозеландской пинии',
  '22',
  'четырёхструнная',
  '1 x P-Стайл + 1 x J-Стайл',
  'Индонезия',
  nan,
  nan,
  nan,
  nan]]

In [22]:
len(values_bass)

600

Проверим, что описание каждого инструмента включает в себя все 17 признаков.

In [23]:
summa = 0
for i in range(len(values_bass)):
    if len(values_bass[i]) != 17:
        summa += 1

In [24]:
summa

0

Создадим датафрейм "description_df", в котором строками будут инструменты, а столбцами - признаки.

In [25]:
description_df = pd.DataFrame(data = np.array(values_bass), columns = features)

In [26]:
description_df

,Форма корпуса,Материал корпуса,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Гриф,Накладка грифа,Количество ладов,Число струн,Типы звукоснимателей,Страна производства,Звукосниматель грифа,Бриджевый звукосниматель,Встроенная электроника,Средний звукосниматель
0,Битлз бас,Клён (мэйпл),Ель,Полуакустическая (Семи Холлоу Бади),Закреплён винтами,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,Хамбакер,КНР,nan,nan,nan,nan
1,Битлз бас,Клён (мэйпл),Ель,Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,Соупбар,КНР,Хофнер Игнишн Стэйпл Никел,Хофнер Игнишн Стэйпл Никел,nan,nan
2,Саундджиэр,Окоуме,nan,Солид Бади,Закреплён винтами,Короткая,"28,63"" (727 мм)",Клён (мэйпл),Древесина новозеландской пинии,22,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,Индонезия,nan,nan,nan,nan
3,"Форма ""Даблкатэвей""",Ясень,"Клён, расщеплённый (неповторимый эффект)",Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Пятислойный клён/орех,Палисандр (розовое дерево),24,пятиструнная,Хамбакер,Индонезия,ESP дизайнд SB-5N,ESP дизайнд SB-5B,Трёхполосный эквалайзер,nan
4,Саундджиэр,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Пятислойный клён/орех,Ятоба,24,четырёхструнная,Хамбакер,Индонезия,Пауэр Спэн дуал койл пикап,Пауэр Спэн дуал койл пикап,Трёхполосный эквалайзер,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Спектор,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Трёхслойный клён,Амарант,24,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,КНР,nan,Спектор Джи,nan,Спектор P
596,Спектор,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Сверхдлинная,"35"" (889 мм)",Трёхслойный клён,Амарант,24,пятиструнная,Хамбакер,КНР,Спектор Хамбакер,Спектор Хамбакер,nan,Спектор P
597,"Форма ""Стинг рэй""",Махагони,nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Клён (мэйпл),Клён (мэйпл),21,четырёхструнная,Форма ММ,Индонезия,nan,Керамический Хамбакер,Двухполосный эквалайзер,nan
598,"Форма ""Стинг рэй""",Махагони,nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Клён (мэйпл),Розовое дерево/палисандр,21,четырёхструнная,Форма ММ,Индонезия,nan,Керамический Хамбакер,Двухполосный эквалайзер,nan


Присоединим полученный датафрейм к названиям инструментов и ценам.

In [27]:
musik_productiv_df = pd.DataFrame({'Наименование' : bass_name_list, 
                                  'Цена' : price_list_stripped})

In [28]:
musik_productiv_df = pd.merge(musik_productiv_df, description_df, how = 'left', left_index=True, right_index = True)

In [29]:
musik_productiv_df

,Наименование,Цена,Форма корпуса,Материал корпуса,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Гриф,Накладка грифа,Количество ладов,Число струн,Типы звукоснимателей,Страна производства,Звукосниматель грифа,Бриджевый звукосниматель,Встроенная электроника,Средний звукосниматель
0,Höfner Ignition Club Bass VSB,25388,Битлз бас,Клён (мэйпл),Ель,Полуакустическая (Семи Холлоу Бади),Закреплён винтами,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,Хамбакер,КНР,nan,nan,nan,nan
1,Höfner Violin Ignition Bass SB,24940,Битлз бас,Клён (мэйпл),Ель,Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,Соупбар,КНР,Хофнер Игнишн Стэйпл Никел,Хофнер Игнишн Стэйпл Никел,nan,nan
2,Ibanez miKro GSRM20 WNF,14711,Саундджиэр,Окоуме,nan,Солид Бади,Закреплён винтами,Короткая,"28,63"" (727 мм)",Клён (мэйпл),Древесина новозеландской пинии,22,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,Индонезия,nan,nan,nan,nan
3,ESP LTD B-205 SM STBLKS,50432,"Форма ""Даблкатэвей""",Ясень,"Клён, расщеплённый (неповторимый эффект)",Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Пятислойный клён/орех,Палисандр (розовое дерево),24,пятиструнная,Хамбакер,Индонезия,ESP дизайнд SB-5N,ESP дизайнд SB-5B,Трёхполосный эквалайзер,nan
4,Ibanez Soundgear SR300E-SVM,25316,Саундджиэр,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Пятислойный клён/орех,Ятоба,24,четырёхструнная,Хамбакер,Индонезия,Пауэр Спэн дуал койл пикап,Пауэр Спэн дуал койл пикап,Трёхполосный эквалайзер,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Spector Performer 4 MBL,32781,Спектор,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Трёхслойный клён,Амарант,24,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,КНР,nan,Спектор Джи,nan,Спектор P
596,Spector Performer 5 BLK,33528,Спектор,Нато (Ньято / Мора),nan,Солид Бади,Закреплён винтами,Сверхдлинная,"35"" (889 мм)",Трёхслойный клён,Амарант,24,пятиструнная,Хамбакер,КНР,Спектор Хамбакер,Спектор Хамбакер,nan,Спектор P
597,Sterling by Music Man StingRay Classic Ray24CA...,51823,"Форма ""Стинг рэй""",Махагони,nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Клён (мэйпл),Клён (мэйпл),21,четырёхструнная,Форма ММ,Индонезия,nan,Керамический Хамбакер,Двухполосный эквалайзер,nan
598,Sterling by Music Man StingRay Classic Ray24CA...,51823,"Форма ""Стинг рэй""",Махагони,nan,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Клён (мэйпл),Розовое дерево/палисандр,21,четырёхструнная,Форма ММ,Индонезия,nan,Керамический Хамбакер,Двухполосный эквалайзер,nan


In [30]:
musik_productiv_df.to_csv('musik_productiv_df', index = False)

## Вывод

Данные с сайта "musik-produktiv.com" получены и представляют собой единый датафрейм. Количество инструментов: 600, количество признаков: 19